In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
# eval_iters = 250
# dropout = 0.2

Using cpu device


In [33]:
with open ('Stock_Exchange.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'æ', '—', '‘', '’', '“', '”', '•', '™']


In [34]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 28, 40, 39, 45, 30, 39, 45, 44,  0,  0,  0,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1, 34, 12,  1, 48, 33, 26, 45,  1, 45, 33,
        30,  1, 44, 45, 40, 28, 36,  1, 30, 49, 28, 33, 26, 39, 32, 30,  1, 34,
        44,  1,  1,  1,  1,  1,  1,  1,  1,  1])


In [35]:
print(chars)
print(string_to_int)
print(int_to_string)
print(encode(text))
print(data)

['\n', ' ', '!', '"', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'æ', '—', '‘', '’', '“', '”', '•', '™']
{'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '_': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62,

In [36]:
n = int(0.8*len(data))
train_data = data [:n]
val_data = data[:n]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([ 31778, 173762,  48693,  61749])
inputs:
tensor([[67, 66, 53, 54, 64, 57,  1, 75],
        [70, 61, 72, 77,  1, 67, 58,  1],
        [53, 70, 57,  1, 55, 53, 64, 64],
        [57, 70, 72, 61, 58, 61, 55, 53]])
targets:
tensor([[66, 53, 54, 64, 57,  1, 75, 60],
        [61, 72, 77,  1, 67, 58,  1, 72],
        [70, 57,  1, 55, 53, 64, 64, 57],
        [70, 72, 61, 58, 61, 55, 53, 72]])


In [37]:
block_size = 40
train_data[:block_size+1]

tensor([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 28, 40, 39, 45])

In [38]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(0)
when input is tensor([0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0, 0]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) target is tenso

In [39]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [40]:
class BigramLanguageModel(nn.Module):
#embedding layer
#The Initialization Method:This method sets up a new instance of our neural network class. The parameter vocab_size is crucial because it tells the model how many unique tokens (e.g., words) it needs to handle. This information is foundational for creating an embedding layer that accurately reflects the vocabulary's size.
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        


TD4;—.D_CFiJ(.zU‘”QOXt0ie$wT”Q)riybz4vdz‘*xX58U(nz£joGAaf1;k%i$“h3sPNXafk5'RZG1n:VfhD™PCOL “Qf%L“ æVmGra“RCVVc7•BSz‘3Qof“hwqw9f%‘HK0h*4x"3%_VL uhL7W-Sb‘x™VjwMq™—F3vUN;YCn$iKb“*Up$ s—0lOL9ZgxV(ggCaaxy$æ‘.3_BGUe”Q_p$PEKAvqToz7O-!XeBSc
'P,4//k!tPNGTdtSb(G14“h—21™PbZ*x’Q"rXp4*J!XcO’aex!t:"48vsDf7x——"74PLmCY:4 —NY%-0—4mxQD_:v‘c0t0p
Z:£M‘FTu WUB—/Vv3:r“;qI$XmA”Qy”S,K6q Wov-pi12Y”QCnM.sCu:•3sVu™d*4vWBqtfKæoAæDNG—!TuxIoc”QhYt blVoc6M“BDdRs£zU2oCt7æRZ(pbkN*ShA3y5hrXiK5•kN
evZqVv7C$w2Hwnn.’9
m—'8‘7‘J™m(gN


In [49]:
# optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

# for iter in range(max_iters):
#     if iter % eval_iters==0:
#         losses = estimate_loss()
#         print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

#     xb, yb = get_batch('train')
#     logits, loss = model.forward(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()
# print(loss.item())
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
desired_steps = [0, 250, 500, 750, 1000]  # List of specific steps you want to print information for

for iter in range(max_iters):
    if iter in desired_steps:  # Check if iter is one of the desired steps
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


tensor([  7921, 124821,  26225,  47450])
tensor([ 44247, 100842,  35884,  25873])
tensor([ 61054, 162467, 132000, 129749])
tensor([146822, 141563, 122176,  89741])
tensor([ 43438,  89459, 149109,  32275])
tensor([ 90552, 100224, 122207,   5181])
tensor([104466,  32675,  82443,  77597])
tensor([155829,  16682,   9813,  84749])
tensor([ 88040, 135416,  95484,  31909])
tensor([ 60882,  92479, 136591,   9439])
tensor([96759, 26837, 98332, 69599])
tensor([ 27152,   1803,  68886, 122725])
tensor([ 90090, 163958, 121826,  95740])
tensor([ 72925,  62889, 120995, 105128])
tensor([ 85318, 123460, 100947, 141258])
tensor([132798,   6599, 104898,   6038])
tensor([16135, 63040, 12981, 43615])
tensor([91746, 92591, 26558, 84220])
tensor([ 46458,  43545, 148606,  50434])
tensor([164835, 147264, 115413, 170113])
tensor([ 38289, 115992,  19442,    210])
tensor([18557, 96370, 41707, 25375])
tensor([19418, 48749, 54454, 43727])
tensor([ 94465,   7725,  62469, 148985])
tensor([102841,  89261,  86887,  545

In [45]:
context = torch.zeros(1, 1, dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


'.zUUBq0w™rs’3y.ts
.0tJ.’7"Rt/HrzxVun3FuBjok;ea$*8fKxd”MY’)mScakux utaU T0Q:H7—q7M*4nU•‘’æ2 e£Nm-F_diiKxEVse£R‘xz’“‘7
..DX;—wnX!8£JN_ZXc'TUpobAZ2TTe!SO-wm”3æR,C'/6*_:oB‘(”jo$•I•/$*WUD_:
!%‘d*2gj7O9A——£bBL8”usAd “.et7f)KJYV-ni"bts'wl5H•9Na$nM*2I)“thA7R-'tNKp$£jxa6L M
Kab$1;ngNcQOy“‘IjKænd-Cr!(PW-;DL.™”xCDX'
)Oj—)“”$d*3e,1DD_vATFixYX—(”’Px.3.V50;)clJj—E
;—4“yES9$j4æ”$*4‘enX'_pA"//ga‘TJwA”6$8 $uiR-yDf8ok *•yR%*i"3x:v3‘me£c0,X br“!—.NmeP,Y'• wuI9a7qE'ea"W5LRr-LmUGmLOI• ayp—1-Zn_p)spBShY8Z!80x'wIz’kB
